# Advanced Deep Learning for NLP - Text based adventure game RL


WIP: Links not assigned yet

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/agents/blob/master/docs/tutorials/1_dqn_tutorial.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/agents/blob/master/docs/tutorials/1_dqn_tutorial.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
</table>

## Google Colab stuff

In [1]:
%%capture
!pip install tf-agents
!pip install textworld
!pip install tensorflow-text

from google.colab import drive
drive.mount("/content/drive")
import os
os.chdir("/content/drive/MyDrive/DeepLearningNLP/")

Or alternatively (for future, when project not private anymore):

In [ ]:
!git clone https://github.com/

Cloning into 'github.com'...
fatal: repository 'https://github.com/' not found


## Imports

In [2]:
from resources import DEFAULT_PATHS
from tests import test_environment_creation

from tf_train_loop import TWTrainer

%load_ext autoreload
%autoreload 2

## Generate games


Generate simple debug game and large dataset of train and eval games.

In [ ]:
os.chdir("/content/drive/MyDrive/DeepLearningNLP/scripts/")

Will create single debug game. This is necessary.

In [ ]:
%%shell
bash ./make_debug_game.sh

Global seed: 2021
Game generated: /content/drive/My Drive/DeepLearningNLP/resources/game_th_lvl2_simple.ulx


Only run this if necessary. Depending on system will take hours.

In [ ]:
%%capture
%%shell
bash ./make_allgames.sh

In [ ]:
os.chdir("/content/drive/MyDrive/DeepLearningNLP/")

## Play a game yourself

In [11]:
os.chdir("/content/drive/MyDrive/DeepLearningNLP/resources/train_games_lvl4/")

In [12]:
!tw-play game_th_lvl4_10.ulx

Using TWInform7.




                    ________  ________  __    __  ________
                   |        \|        \|  \  |  \|        \
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$
                      | $$   | $$__     \$$\/  $$   | $$
                      | $$   | $$  \     >$$  $$    | $$
                      | $$   | $$$$$    /  $$$$\    | $$
                      | $$   | $$_____ |  $$ \$$\   | $$
                      | $$   | $$     \| $$  | $$   | $$
                       \$$    \$$$$$$$$ \$$   \$$    \$$
              __       __   ______   _______   __        _______
             |  \  _  |  \ /      \ |       \ |  \      |       \
             | $$ / \ | $$|  $$$$$$\| $$$$$$$\| $$      | $$$$$$$\
             | $$/  $\| $$| $$  | $$| $$__| $$| $$      | $$  | $$
             | $$  $$$\ $$| $$  | $$| $$    $$| $$      | $$  | $$
             | $$ $$\$$\$$| $$  | $$| $$$$$$$\| $$      | $$  | $$
             | $$$$  \$$$$| $$__/ $$| $$  | $$| $$_____ | $

In [ ]:
os.chdir("/content/drive/MyDrive/DeepLearningNLP/")

# Test environment

In [3]:
test_environment_creation()

Starting new game.
{'score': 0, 'done': False, 'won': False, 'lost': False, 'obs': "-= Parlor =-\nYou've just shown up in a parlor.\n\n\n\nThere is an exit to the east. Don't worry, it is unblocked.\n\nThere is a blueberry and a controller on the floor.\n\n", 'description': "-= Parlor =-\nYou've just shown up in a parlor.\n\n\n\nThere is an exit to the east. Don't worry, it is unblocked.\n\nThere is a blueberry and a controller on the floor.", 'inventory': 'You are carrying nothing.', 'admissible_commands': ['examine blueberry', 'examine controller', 'go east', 'inventory', 'look', 'take blueberry', 'take controller'], 'entities': ['blueberry', 'controller', 'north', 'south', 'east', 'west'], 'objective': "It's time to explore the amazing world of TextWorld! Your task for today is to lift the controller from the floor of the parlor."}
Doing: take clove
{'score': 0, 'done': False, 'won': False, 'lost': False, 'obs': "You can't see any such thing.\n", 'description': "-= Parlor =-\nYou've

# Train

Set rewards for training.

In [3]:
REWARD_DICT = {
    "win_lose_value": 100,
    "max_loop_pun": 0,
    "change_reward": 1,
    "useless_act_pun": 1,
    "cmd_in_adm": 1,
}

Try overfitting on one debug game (correct command "take x" will immediately win or lose the game).
Depending on whether random agent finds correct WIN command, number of iterations will be enough or not.


In [ ]:
DEFAULT_HP = {
    "learning_rate": 1e-3,
    "initial_collect_steps": 3000,
    "collect_steps_per_iteration": 1,
    "replay_buffer_max_length": 100000,
    # large values lead to OOM with bert policy
    "batch_size": 32,
    "num_eval_episodes": 1,
    "game_gen_buffer": 25,
    "num_eval_games": 5,
}
trainer = TWTrainer(
    reward_dict=REWARD_DICT,
    hpar=DEFAULT_HP,
    debug=False,
    biased_buffer=True,
    # embedding into fc is default policy
    # agent_label="FCPolicy",
    agent_label="BertPolicy",
)
eval_scores = trainer.train(
    num_iterations=2000,
    log_interval=100,
    eval_interval=100,
    game_gen_interval=500,
    plot_avg_ret=True,
)

Activate TensorBoard logging.

In [4]:
pathdir = DEFAULT_PATHS["path_logdir"]
%load_ext tensorboard
%tensorboard --logdir $pathdir

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 10017.

Train on all four levels with (simple) Embedding-FC policy agent

In [5]:
DEFAULT_HP = {
    "learning_rate": 1e-3,
    "initial_collect_steps": 10000,
    "collect_steps_per_iteration": 1,
    "replay_buffer_max_length": 100000,
    "batch_size": 64,
    "num_eval_episodes": 1,
    "game_gen_buffer": 25,
    "num_eval_games": 10,
}

trainer = TWTrainer(
    env_dir="train_games_lvl1",
    reward_dict=REWARD_DICT,
    hpar=DEFAULT_HP,
    debug=False,
    biased_buffer=True,
    # embedding into fc is default policy
    # agent_label="FCPolicy",
    agent_label="BertPolicy",
)

In [ ]:
print(f"Starting with lvl: 1 \n")

eval_scores = trainer.train(
    num_iterations=5000,
    log_interval=250,
    eval_interval=500,
    game_gen_interval=500,
    rndm_fill_replay=True,
    plot_avg_ret=True,
)

print(f"Changing to next lvl: 2 \n")

trainer.change_env_dir(f"train_games_lvl2")
eval_scores = trainer.train(
    num_iterations=5000,
    log_interval=250,
    eval_interval=500,
    game_gen_interval=500,
    continue_training=True,
    rndm_fill_replay=True,
    plot_avg_ret=True,
)

print(f"Changing to next lvl: 3 \n")

trainer.change_env_dir(f"train_games_lvl3")
eval_scores = trainer.train(
    num_iterations=5000,
    log_interval=250,
    eval_interval=500,
    game_gen_interval=500,
    continue_training=True,
    rndm_fill_replay=True,
    plot_avg_ret=True,
)

print(f"Changing to next lvl: 4 \n")

trainer.change_env_dir(f"train_games_lvl4")
eval_scores = trainer.train(
    num_iterations=10000,
    log_interval=250,
    eval_interval=500,
    game_gen_interval=500,
    continue_training=True,
    rndm_fill_replay=True,
    plot_avg_ret=True,
)


Starting with lvl: 1 



## Hyper parameter search

Simple hyper parameter search.

CODE OUT OF DATE. Still used old train method.

In [ ]:
%%capture
!pip install optuna
import optuna

In [ ]:
def objective(trial):
    """"""

    hp_dict = {
      "num_iterations": 2000,
      "learning_rate": trial.suggest_loguniform("lr", 1e-4, 1e-2),
      "initial_collect_steps": 10000,
      "collect_steps_per_iteration": 1,
      "replay_buffer_max_length": 100000,
      "batch_size": trial.suggest_int("batch_size", 16, 256),
      "log_interval": 500,
      "num_eval_episodes": 1,
      "eval_interval": 100,
    }

    REWARD_DICT = {
        "win_lose_value": 100,
        "max_loop_pun": 1,
        "change_reward": 1,
        "useless_act_pun": 1,
        "cmd_in_adm": 1,
    }

    eval_scores = train.main(plot_avg_ret=True, debug=False, reward_dict=REWARD_DICT, **hp_dict)
    # eval_scores = np.delete(eval_scores, 0)

    return max(eval_scores)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)
print(study.best_trial)

In [ ]:
print(study.best_trial)

FrozenTrial(number=0, values=[203.0], datetime_start=datetime.datetime(2021, 7, 30, 11, 46, 26, 924586), datetime_complete=datetime.datetime(2021, 7, 30, 11, 50, 2, 140224), params={'lr': 0.004015582546958087, 'batch_size': 89}, distributions={'lr': LogUniformDistribution(high=0.01, low=0.0001), 'batch_size': IntUniformDistribution(high=256, low=16, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=0, state=TrialState.COMPLETE, value=None)
